In [2]:
import os
os.getcwd()

'/home/onyxia/work/hackathon-un-2022/notebooks'

In [4]:
os.chdir("..")
import utils.functions as fc

In [21]:
from importlib import reload
fc = reload(fc)

In [17]:
ship_data_enriched = fc.create_ship_data_enriched()

In [22]:
AIS = fc.read_ais_parquet()